In [1]:
import pandas as pd

titanic_data = pd.read_csv('datasets/Titanic-Dataset.csv', quotechar ='"')
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
titanic_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
titanic_data['Sex'] = le.fit_transform(titanic_data['Sex'].astype(str))
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [4]:
titanic_data = pd.get_dummies(titanic_data, columns=['Embarked'])
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,False,False,True
1,1,1,0,38.0,1,0,71.2833,True,False,False
2,1,3,0,26.0,0,0,7.9250,False,False,True
3,1,1,0,35.0,1,0,53.1000,False,False,True
4,0,3,1,35.0,0,0,8.0500,False,False,True


In [5]:
titanic_data[titanic_data.isnull().any(axis=1)]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
5,0,3,1,NaN,0,0,8.4583,False,True,False
17,1,2,1,NaN,0,0,13.0000,False,False,True
19,1,3,0,NaN,0,0,7.2250,True,False,False
26,0,3,1,NaN,0,0,7.2250,True,False,False
28,1,3,0,NaN,0,0,7.8792,False,True,False
...,...,...,...,...,...,...,...,...,...,...
859,0,3,1,NaN,0,0,7.2292,True,False,False
863,0,3,0,NaN,8,2,69.5500,False,False,True
868,0,3,1,NaN,0,0,9.5000,False,False,True
878,0,3,1,NaN,0,0,7.8958,False,False,True


In [6]:
titanic_data = titanic_data.dropna()

In [15]:
from sklearn.cluster import MeanShift

analyzer = MeanShift(bandwidth=30)
analyzer.fit(titanic_data)

,bandwidth,30
,seeds,None
,bin_seeding,False
,min_bin_freq,1
,cluster_all,True
,n_jobs,None
,max_iter,300


In [16]:
from sklearn.cluster import estimate_bandwidth

estimate_bandwidth(titanic_data)


np.float64(30.446971266796215)

In [17]:
import numpy as np

labels = analyzer.labels_
np.unique(labels)

array([0, 1, 2, 3, 4])

In [18]:
titanic_data['cluster_group'] = np.nan
data_length = len(titanic_data)
for i in range(data_length):
    titanic_data.iloc[i,titanic_data.columns.get_loc('cluster_group')] = labels[i]

In [19]:
titanic_data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,cluster_group
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000
mean,0.406162,2.236695,0.634454,29.699118,0.512605,0.431373,34.694514,0.315126
std,0.491460,0.838250,0.481921,14.526497,0.929783,0.853289,52.918930,0.690647
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,20.125000,0.000000,0.000000,8.050000,0.000000
50%,0.000000,2.000000,1.000000,28.000000,0.000000,0.000000,15.741700,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,1.000000,33.375000,0.000000
max,1.000000,3.000000,1.000000,80.000000,5.000000,6.000000,512.329200,4.000000


In [20]:
titanic_cluster_data = titanic_data.groupby('cluster_group').mean()
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
cluster_group,,,,,,,,,,
0.0,0.336918,2.525090,0.679211,28.256720,0.439068,0.370968,15.434139,0.121864,0.046595,0.831541
1.0,0.611111,1.296296,0.527778,36.148148,0.814815,0.500000,65.622688,0.333333,0.018519,0.629630
2.0,0.733333,1.000000,0.366667,32.430667,0.600000,0.866667,131.183883,0.500000,0.000000,0.500000
3.0,0.733333,1.000000,0.266667,30.333333,1.000000,1.333333,239.991940,0.533333,0.000000,0.466667
4.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000


In [21]:
titanic_cluster_data['Counts'] = pd.Series(titanic_data.groupby('cluster_group').size())
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Counts
cluster_group,,,,,,,,,,,
0.0,0.336918,2.525090,0.679211,28.256720,0.439068,0.370968,15.434139,0.121864,0.046595,0.831541,558
1.0,0.611111,1.296296,0.527778,36.148148,0.814815,0.500000,65.622688,0.333333,0.018519,0.629630,108
2.0,0.733333,1.000000,0.366667,32.430667,0.600000,0.866667,131.183883,0.500000,0.000000,0.500000,30
3.0,0.733333,1.000000,0.266667,30.333333,1.000000,1.333333,239.991940,0.533333,0.000000,0.466667,15
4.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000,3


In [22]:
titanic_data[ titanic_data['cluster_group'] == 1].describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,cluster_group
count,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.0
mean,0.611111,1.296296,0.527778,36.148148,0.814815,0.500000,65.622688,1.0
std,0.489771,0.645028,0.501555,14.919607,1.086434,0.971558,15.634315,0.0
min,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,34.654200,1.0
25%,0.000000,1.000000,0.000000,24.000000,0.000000,0.000000,52.554200,1.0
50%,1.000000,1.000000,1.000000,35.000000,1.000000,0.000000,65.000000,1.0
75%,1.000000,1.000000,1.000000,48.000000,1.000000,1.000000,78.937500,1.0
max,1.000000,3.000000,1.000000,71.000000,5.000000,6.000000,93.500000,1.0


In [23]:
titanic_data[ titanic_data['cluster_group'] == 2].describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,cluster_group
count,30.000000,30.0,30.000000,30.000000,30.000000,30.000000,30.000000,30.0
mean,0.733333,1.0,0.366667,32.430667,0.600000,0.866667,131.183883,2.0
std,0.449776,0.0,0.490133,15.638593,0.563242,0.899553,19.170620,0.0
min,0.000000,1.0,0.000000,0.920000,0.000000,0.000000,106.425000,2.0
25%,0.250000,1.0,0.000000,22.000000,0.000000,0.000000,113.275000,2.0
50%,1.000000,1.0,0.000000,35.500000,1.000000,1.000000,134.075000,2.0
75%,1.000000,1.0,1.000000,40.750000,1.000000,2.000000,151.550000,2.0
max,1.000000,1.0,1.000000,58.000000,2.000000,2.000000,164.866700,2.0
